In [50]:
from numpy import array
from numpy import hstack
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import numpy as np
import os

In [51]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [52]:
#Returns array of values with True_data(available frames)
def Missing_Frames(data_info):
    True_Data = np.empty((0, 2), int)
    From = 0
    false_found = 0
    last_true0 = 0
    last_true1 = 0
    for index, row in data_info.iterrows():
        #print(row['Mocap_Data_Exist'])
        if(row['Video_Data_Exist'] == False):
            if(false_found == 0):
                True_Data = np.append(True_Data, np.array([[From, index-1]]), axis=0)
            false_found = 1
            From = index+1
            last_true0 = index
        else:
            false_found = 0
            last_true1 = index
    True_Data = np.append(True_Data, np.array([[last_true0+1, last_true1]]), axis=0)
    return True_Data

In [ ]:
def LSTM_Model(X,y,n_steps,n_features):
    #model definition and fit
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=400, verbose=0)

In [53]:
#Timeseries Prediction Function
def PredictTimeSeries(x_input,n_times,n_steps,n_features):
    # demonstrate prediction for next 10 days
    #x_input = array([[ -54,1618,65,-35,1528,-20,-147,1536,83],
     #                [-55,1617,63,-36,1526,-21,147,1535,82], 
      #               [-55,1617,62,-36,1525,-22,148,1535,81], 
       #              [-55,1617,61,-36,1535,-22,-148,1535,80]])

    temp_input=list(x_input)
    lst_output=[]
    i=0
    while(i<n_times):
        #this length is same as steps
        if(len(temp_input)>n_steps):
            #print("temp input 3 ",  temp_input)
            x_input=array(temp_input[1:])
            #print("temp input 4 ",  x_input)
            #print("{} day input {}".format(i,x_input))
            #print(x_input)
            x_input = x_input.reshape((1, n_steps, n_features))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
            #print("{} day output {}".format(i,yhat))
            temp_input.append(yhat[0])
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.append(yhat[0])
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            #print(yhat[0])
            #print("temp input 1 ",  temp_input)
            temp_input.append(yhat[0])
            #print("temp input 2 ",  temp_input)
            lst_output.append(yhat[0])
            i=i+1
    return lst_output

In [57]:
#Get list of info csv form directory
def fn_getfilenames(dir): 
    return os.listdir(dir)

dirVideoCap="data-csv"
dirInfo = "data"
file_list = fn_getfilenames(dirInfo)
infofile_list = []
for file in file_list:
    if "_info.csv" in file:
        infofile_list.append(file)
infofile_list.sort()
infofile_list

['s037t01_info.csv',
 's037t02_info.csv',
 's037t03_info.csv',
 's037t04_info.csv',
 's037t06_info.csv',
 's038t07_info.csv',
 's039t03_info.csv',
 's040t01_info.csv',
 's040t04_info.csv',
 's041t01_info.csv',
 's041t05_info.csv',
 's042t06_info.csv',
 's043t02_info.csv',
 's044t01_info.csv',
 's044t03_info.csv',
 's046t05_info.csv',
 's047t02_info.csv',
 's047t04_info.csv',
 's048t06_info.csv',
 's049t01_info.csv',
 's049t06_info.csv',
 's050t01_info.csv',
 's050t06_info.csv',
 's052t02_info.csv',
 's052t06_info.csv',
 's054eo_info.csv',
 's054t04_info.csv',
 's054t05_info.csv',
 's054t06_info.csv',
 's055eo_info.csv',
 's055t03_info.csv',
 's056t05_info.csv',
 's057t05_info.csv',
 's057t06_info.csv',
 's058t03_info.csv',
 's059t04_info.csv',
 's063t02_info.csv',
 's064t03_info.csv',
 's070t04_info.csv']

In [55]:
# choose a number of time steps for Time Series
n_steps = 10

In [56]:
#CSV Pixel - Prediction
print("Total files to process: " + str(len(infofile_list)))
predicted_all = pd.DataFrame() #to store the predicted datavalues
file_count=1

#for all Files
for file in infofile_list:
    #missing frames dataset with True/False
    data_info = pd.read_csv(dirInfo + "/" + file)
    #print(dirInfo + "/" + file)
    TrueFalse_Data = Missing_Frames(data_info)
      
    VideoCap_file = file.replace("_info.csv", ".csv")
    #print("\nProcessing file : " + VideoCap_file + " ...")
    print("\n" + str(file_count) + ".Processing file : " + VideoCap_file + " ...")
    print("Missing frame(True) summary of this file")
    print(TrueFalse_Data)
  
    #print(dirVideoCap + "/" + VideoCap_file)
    data = pd.read_csv(dirVideoCap + "/" + VideoCap_file)
    data = data.iloc[:,11:]
    #print(data.head(1))
    #print(list(data.columns))
    dataset = data.to_numpy()
    #print(dataset)
    rowIndex=0
    #True_Frame = data.iloc[TrueFalse_Data[0][0]:TrueFalse_Data[rowIndex][1]+1,]
    #if 3 entries in TrueFalse_Data, only 2 missing sections
    print("There are " + str(len(TrueFalse_Data)-1) +  " missing frames in the file")
    #print(len(TrueFalse_Data))
    while(rowIndex < len(TrueFalse_Data)-1):
        #print(TrueFalse_Data[rowIndex][0])
        #print(TrueFalse_Data[rowIndex][1])
        
        True_Frame = data.iloc[TrueFalse_Data[rowIndex][0]:TrueFalse_Data[rowIndex][1]+1,]
        #True_Frame_Head = data.iloc[TrueFalse_Data[rowIndex][0]:TrueFalse_Data[rowIndex][1]+1,]
        
        #print(True_Frame)
        #if(rowIndex > 0 ):
        #    True_Frame = pd.concat([predicted_df,True_Frame], ignore_index=True)
        #Convert to Array
        dataset = True_Frame.to_numpy()
        #print(dataset)
        
        # convert into input/output
        X, y = split_sequences(dataset, n_steps)
        #print(X.shape, y.shape)
        
        #RNN - LSTM
        print("Training LSTM Model ...")
        # the dataset knows the number of features, e.g. 2
        n_features = X.shape[2]
        # define and fit model
        #LSTM_Model(X,y,n_steps,n_features)
        
        model = Sequential()
        model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(LSTM(100, activation='relu'))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse')
        # fit model
        #model.fit(X, y, epochs=400, verbose=0)
        model.fit(X, y, epochs=50, verbose=0)
        
        # demonstrate prediction
        x_input = X[len(X)-1]
        #print(x_input)
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        #print("Model output for an input...")
        #print(yhat)
        
        print("Predicting Missing frame " + str(rowIndex+1) + " ...")
        #Prediction 
        #print(TrueFalse_Data[rowIndex][1]+1)
        #print(TrueFalse_Data[rowIndex+1][0]-1)
        MissingFramesCount = (TrueFalse_Data[rowIndex+1][0]-1) - (TrueFalse_Data[rowIndex][1])
        lst_output = PredictTimeSeries(x_input,MissingFramesCount,n_steps,n_features)
        #print(round(pd.DataFrame(lst_output)))
        predicted_df = round(pd.DataFrame(lst_output))
        #print(predicted_df)
        #predicted_df.set_axis(['HeadTop_X', 'HeadTop_Y', 'HeadTop_Z', 'HeadFront_X', 'HeadFront_Y','HeadFront_Z','HeadSide_X','HeadSide_Y','HeadSide_Z'], axis='columns', inplace=True)
        predicted_df.set_axis(data.columns, axis='columns', inplace=True)
        #print(predicted_df)
        predicted_all = pd.concat([predicted_all, predicted_df], ignore_index=True)
        #if(rowIndex == 0):
        #    True_Plus_Predicted_Frame = pd.concat([True_Frame, predicted_df], ignore_index=True)
        #else:
        #    True_Plus_Predicted_Frame = pd.concat([True_Plus_Predicted_Frame, True_Frame, predicted_df], ignore_index=True)


        print("Missing frame " + str(rowIndex+1) + " completed.\n")
        rowIndex=rowIndex+1
    #file_name = "FD_Pred/" + file.replace("_info.csv", "_pred_video.csv")
    #predicted_all.to_csv(file_name,index=False)
    file_count = file_count + 1
    print(predicted_all.shape)
#file_name = "FD_Pred/" + file.replace("_info.csv", "_pred_video.csv")
file_name = "Pred/bbdc_pred_video.csv" 

#if pixel goes above 255 in timeseries prediction, replace with all >255 values with 255
predicted_all[predicted_all>255] = 255
#if pixel goes below zero(-), replace with all < 0 values with 0
predicted_all[predicted_all < 0] = 0

predicted_all.to_csv(file_name,index=False)
print(predicted_all.shape)

print("\nAll files are Completed")

Total files to process: 39

Processing file : s070t04.csv ...
Missing frame(True) summary of this file
[[   1 5438]]
There are 0 missing frames in the file
(0, 0)

Processing file : s049t01.csv ...
Missing frame(True) summary of this file
[[   0  601]
 [ 739  902]
 [1067 1203]
 [1213 1856]]
There are 3 missing frames in the file
Training LSTM Model ...
Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

(310, 15360)

Processing file : s057t05.csv ...
Missing frame(True) summary of this file
[[   1 4846]]
There are 0 missing frames in the file
(310, 15360)

Processing file : s040t01.csv ...
Missing frame(True) summary of this file
[[   0  601]
 [ 671  902]
 [ 989 2106]
 [2174 2407]
 [2435 3310]
 [3362 3912]
 [3931 4213]
 [4258 5116]
 [5289 5417]
 [5536 5718]
 [5882 6320]
 [6380 6621]
 [6697 7520]]
There are 12 missin

Missing frame 9 completed.

Training LSTM Model ...
Predicting Missing frame 10 ...
Missing frame 10 completed.

Training LSTM Model ...
Predicting Missing frame 11 ...
Missing frame 11 completed.

(5271, 15360)

Processing file : s037t02.csv ...
Missing frame(True) summary of this file
[[   0  902]
 [ 940 1203]
 [1280 1504]
 [1530 2106]
 [2149 2407]
 [2505 2708]
 [2755 3611]
 [3690 4436]]
There are 7 missing frames in the file
Training LSTM Model ...
Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

Training LSTM Model ...
Predicting Missing frame 6 ...
Missing frame 6 completed.

Training LSTM Model ...
Predicting Missing frame 7 ...
Missing frame 7

Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

(12080, 15360)

Processing file : s044t01.csv ...
Missing frame(True) summary of this file
[[   0  300]
 [ 516 2106]
 [2153 2407]
 [2486 2708]
 [2805 3009]
 [3045 3310]
 [3353 3611]
 [3671 3912]
 [4051 4213]
 [4300 5477]]
There are 9 missing frames in the file
Training LSTM Model ...
Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

Training LSTM Model ...
Predicting Missing fram

Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

Training LSTM Model ...
Predicting Missing frame 6 ...
Missing frame 6 completed.

Training LSTM Model ...
Predicting Missing frame 7 ...
Missing frame 7 completed.

(18923, 15360)

Processing file : s054t06.csv ...
Missing frame(True) summary of this file
[[   0  300]
 [ 382  601]
 [ 745  902]
 [ 983 1203]
 [1366 1805]
 [1999 3009]
 [3209 4213]
 [4295 4514]
 [4532 5116]
 [5118 5718]
 [5823 6019]
 [6157 6927]]
There are 11 missing frames in the file
Training LSTM Model ...
Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model 

### Convert Pixel to Image

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
#dfpred = pd.read_csv("Pred/s049t01_pred_video.csv")
dfpred = pd.read_csv("Pred/bbdc_pred_video.csv")

In [ ]:
rowIndex=0
print(len(dfpred))
#while (rowIndex < len(dfpred)):
while (rowIndex < 2):
    df = dfpred.loc[rowIndex]
    print(rowIndex)
    print(df.shape)
    lst = df.to_numpy() 
    #print(lst.shape)
    array = np.array(lst, dtype=np.uint8)
    #print(array.shape)

    #new_image = Image.fromarray(array)
    #new_image.save('new4.png')
    imgname = "img"+str(rowIndex)
    imgname = Image.fromarray(array.reshape(96,160)) 
    #print(imgname)
    plt.imshow(imgname, interpolation='nearest')
    #img =  str(rowIndex) + str(".jpg")
    img =  str("Images/") + str(rowIndex) + str(".jpg")
    plt.savefig(img,format="jpg")
    #plt.plot(plt)
    rowIndex = rowIndex+1
    #print(plt.show)

### Convert Image to Video

In [ ]:
import cv2
import numpy as np
import os
from os.path import isfile, join

In [ ]:
#Get list of info csv form directory
def fn_getimagenames(dir): 
    return os.listdir(dir)

#dirVideoCap="data-csv-test"
#dirInfo = "data_test"
#file_list = fn_getfilenames(dirInfo)
#infofile_list = []
#for file in file_list:
    #if "_info.csv" in file:
        #infofile_list.append(file)
#infofile_list

In [ ]:
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    #files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    files = fn_getimagenames(pathIn)
    files.remove(".DS_Store")
    #for sorting the file names properly
    files.sort()#key = lambda x: int(x[5:-4]))
    
    print(files)
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        print(filename)
        print(img)
        height, width, layers = img.shape
        size = (width,height)
        #print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
def main():
    pathIn= 'Images/'
    pathOut = 'video.mp4'
    #fps = 25.0
    fps = 30
    convert_frames_to_video(pathIn, pathOut, fps)
if __name__=="__main__":
    main()